### Домашнее задание к лекции «Продвинутый pandas 2»

In [56]:
import pandas as pd

**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [57]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [58]:
users_min = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': 'min'})
users_min = users_min.rename(columns={'timestamp': 'timestamp_min', 'rating': 'mark_count'})

users_max = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': 'max'})
users_max = users_max.rename(columns={'timestamp': 'timestamp_max', 'rating': 'mark_count'})

users_min_filtered = users_min [ (users_min['mark_count'] >= 100)]
users_max_filtered = users_max [ (users_min['mark_count'] >= 100)]

In [59]:
# Проверка таблиц
print(len(users_min_filtered), sum(users_min_filtered['mark_count']))
print(len(users_max_filtered), sum(users_max_filtered['mark_count']))
# users_min.head()

263 80987
263 80987


In [60]:
users_max_filtered = users_max_filtered.loc[:, ['timestamp_max']]

In [61]:
joined = users_min_filtered.join(users_max_filtered, how='left')
joined['timestamp_min'] = pd.to_datetime(joined['timestamp_min'], unit='s')
joined['timestamp_max'] = pd.to_datetime(joined['timestamp_max'], unit='s')
joined['lifetime'] = joined['timestamp_max'] - joined['timestamp_min'] 
joined.head()

,mark_count,timestamp_min,timestamp_max,lifetime
userId,,,,
4,204,2000-02-05 19:25:14,2000-02-08 03:57:54,2 days 08:32:40
5,100,2006-11-12 23:10:44,2006-11-12 23:45:45,0 days 00:35:01
8,116,2006-07-31 23:42:20,2006-08-01 23:22:07,0 days 23:39:47
15,1700,2001-08-16 04:47:19,2016-07-24 03:25:35,5455 days 22:38:16
17,363,2005-09-23 09:43:07,2005-09-23 11:57:20,0 days 02:14:13


In [62]:
# Ответ задачи. 

joined['lifetime'].mean()

Timedelta('455 days 05:29:08.566539')

In [63]:
# Последний блок чуть короче, но с вопросом

joined = users_min_filtered.join(users_max_filtered, how='left')
joined['lifetime'] = joined['timestamp_max'] - joined['timestamp_min'] 
mean_time = joined['lifetime'].mean()

mean_time
#А как можно это число преобразовать теперь в дни/часы/минуты?


39331748.56653992

**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [64]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [65]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [66]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [67]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [68]:
# Объединяем данные по всем таблицам Left Join, за левую таблицу берем базу клиентов
# Таблица с адресами клиентов
client_revenue = client_base.merge(
            rzd, how='left', on='client_id').merge(
            auto, how='left', on='client_id').merge(
            air, how='left', on='client_id').fillna('')

client_revenue.head(10)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093,,
1,112,Энтузиастов 8а,2810,,
2,113,Левобережная 1а,10283,57483,
3,114,Мира 14,5774,83,
4,115,ЗЖБИиДК 1,981,912,81
5,116,Строителей 18,,4834,4
6,117,Панфиловская 33,,98,13
7,118,Мастеркова 4,,,173


In [69]:
# Таблица без адресов клиентов

no_address = client_revenue[['client_id', 'rzd_revenue', 'auto_revenue', 'air_revenue']]

no_address.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093,,
1,112,2810,,
2,113,10283,57483,
3,114,5774,83,
4,115,981,912,81


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

# Решение

- Так как широта и долгота фиксируются в момент визита пользователя в приложение, то у нас должна быть связка каждого визита с конкретными данными о широте и долготе. 
- Не на каждый визит приходится покупка, поэтому общая таблица с покупками, визитами и парметрами ширины и долготы будет выглядеть следующим образом. Если нужно разделить покупки по категориям, то можно добавить несколько столбцов с разными категориями покупок.

In [71]:
data = pd.DataFrame(
    {
        'user_id': [115, 116, 115, 118],
        'Coordinates': [(55.75399400, 37.62209300), (55.75399400, 37.62209300), (55.75399400, 37.62209300), (55.75399400, 37.62209300)],
        'Purchases': [0, 2, 1, 0],
        'Revenue': ['0', '$65', '$30', '0'],
        'Costs': ['0', '$33', '$12', '0'],
        'ROI': ['0', '96%', '150%', '0'],
        'Date': ['2006-11-12 23:45:45', '2006-07-31 23:42:20', '2006-08-01 23:22:07', '2016-07-24 03:25:35']
    }
)

data

,user_id,Coordinates,Purchases,Revenue,Costs,ROI,Date
0,115,"(55.753994, 37.622093)",0,0,0,0,2006-11-12 23:45:45
1,116,"(55.753994, 37.622093)",2,$65,$33,96%,2006-07-31 23:42:20
2,115,"(55.753994, 37.622093)",1,$30,$12,150%,2006-08-01 23:22:07
3,118,"(55.753994, 37.622093)",0,0,0,0,2016-07-24 03:25:35


- Можно посчитать сколько визитов происходит до покупки и время между первым визитом и покупкой. 
    На основе этой информации можно настраивать кампании ремаркетинга или выделять группы пользователей для различных email рассылок. 
- В задаче не уточняется тип бизнеса. Предположим, что это приложение доставки еды. На основе данных о координатах пользователя можно располагать пункты сборки заказов, чтобы сократить скорость доставки заказов.
- Если из какого-то региона достаточно много визитов, но нет покупок, то можно поставить вопрос: Почему и как можно улучшить ситуацию?
- Если какие-то районы приносят больше всего прибыли, стоит усилить рекламную кампанию в этом регионе.

На самом деле не уверена, что правильно поняла вопрос задачи.